Workflow:
- Open the rectified image set
- Perform BackgroundRemoval (createBackgroundSubtractorKNN)
- Identify the Center of Gravity of the Object (Kalman)
- Draw Current Box (Kalman)
- Draw Prediction (Kalman)

In [5]:
import cv2
import numpy as np
import imutils
import glob
import matplotlib.pyplot as plt
import os
import time

Kalman filtering, also known as linear quadratic estimation (LQE), is an algorithm that uses a series of measurements observed over time, including statistical noise and other inaccuracies, and produces estimates of unknown variables that tend to be more accurate than those based on a single measurement alone.

Used to estimate states based on linear dynamical systems in state space format. The evolution from process k-1 to k is given by:

State vector x_{k-1} and the x_{k} the prediction.

$x_k=Fx_{k-1}+Bu_{k-1}+w_{k-1}$

- F: is the transition matrix
- B: Control input matrix
- w: Process noise vector (GAUSSIAN, with Q covariance)

The Process and Measurement/observation model are paired following: 

$z_k=Hx_k+v_k$

- z is the measurement vector
- H is the measurement matrix
- v is the noise vector Gaussian (0,R)

The role of the Kalman filter is to provide estimate of xk at time k, given the initial estimate of x0, the series of measurement, z1,z2,…,zk and the information of the system described by F , B , H , Q , and  R


Kalman filtering has 2 steps:

- Propagation (Prediction)

- Measuremnet (Update)


In [6]:
def Kalman(vars,dim):

    PredVec=vars*dim

    kalman = cv2.KalmanFilter(PredVec, dim) #6,3 Which states for position and velocity in 3D (x,y,z,vx,vy,vz)

    kalman.measurementMatrix = np.array([[1,0,0,0,0,0],             #Z
                                         [0,0,1,0,0,0],
                                         [0,0,0,0,1,0]], np.float32)


    kalman.transitionMatrix = np.array([[1,1,0,0,0,0],              #F
                                        [0,1,0,0,0,0],
                                        [0,0,1,1,0,0],
                                        [0,0,0,1,0,0],
                                        [0,0,0,0,1,1],
                                        [0,0,0,0,0,1]], np.float32)
                                        
    kalman.processNoiseCov = np.eye(6, dtype=np.float32) * 0.03
    kalman.measurementNoiseCov = np.eye(3, dtype=np.float32) * 0.06

    prediction = np.zeros((PredVec,1), np.float32)

    return prediction, kalman

BACKGROUND REMOVAL, since the background is static, the optimal solution is to perform "Frame Difference Method"
https://www.codetd.com/en/article/10747516

In [7]:
def BkgRmv(sel=1):
    if sel == 0:
        print("BackGroundSubstractor KNN")
        fgbg = cv2.createBackgroundSubtractorKNN(history=1000, dist2Threshold=1000, detectShadows=False)
    elif sel== 1:
        print("BackGroundSubstractor Gaussian mixture model")
        fgbg = cv2.createBackgroundSubtractorMOG2()
    elif sel == 2:
        print(" BackGroundSubstractor Geometric Multigrid")
        print(" NEEDS TO BEINSTALLED SEPARATEDLY")
        #fgbg = cv2.bgsegm.createBackgroundSubtractorGMG()
    else: 
        print("Frame Difference method")
        """"
        if frameNum == 1:
    	bgFrame = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
        elif frameNum > 1:
            foreFrame = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
            foreFrame = cv2.absdiff(foreFrame, bgFrame)
            _, thresh = cv2.threshold(foreFrame, 30, 255, cv2.THRESH_BINARY)
            gaussian = cv2.GaussianBlur(thresh, (3, 3), 0)
            cv2.imshow('gaussian', foreFrame)
        """""
    return fgbg
    

The depth function implements the computation, based on the knowledge of the disparity of the 2 pairs of pixels in the rectified image planes. 

The **baseline**  and the **focal lenght** are parameters of the function which condition the output.

Based on the assumption that the images are undistorted, we assume the rectified stereo vision case in which both image planes are horizontal and parallel. Therefore the epipolar lines are parallel and this creates a powerful set of constrains to find the corresponding point on the right image.

The point is recognised by the disparity of a sliding window through the epipolar line. The lowest Sum of Absolute Difference will determine the right point on the left.

Once the point has been found on the right image, disparity can be computed as the difference of the horizontal distances between the point in each image to the center of the image.

Depth can be computed from the disparity simply applying the formula:

In [8]:
def sum_abs_diff(image1, image2):
    image1 = image1.astype('int32')
    image2 = image2.astype('int32')
    sad = 0
    
    if image1.shape == image2.shape:
        diff = image1 - image2
        sad = np.sum(np.absolute(diff))
    else:
        sad = -1
    return sad

def scan_line(span, template, search_col_min, search_col_max):
    min_place = -1
    min_value = float('inf')
    
    for i in range(search_col_min, search_col_max):
        diff = sum_abs_diff(span[:, i:i + span.shape[0]], template)
        if diff < min_value:
            min_value = diff
            min_place = i
    return (min_place, min_value)

def Depth(leftGrayImg, rightGrayImg, positionOnLeft, halfWindow, baseline = 120, focalLength = 700):
    """
    The function takes 6 arguments:
    - The 2 images that are to be used to compute the triangulation,
    - The position of the pixel in the left image,
    - The size of the window to prompt in the template which is also the size of the Sliding window.
    """
        
    template = leftGrayImg[positionOnLeft[0]-halfWindow:positionOnLeft[0]+halfWindow,\
                           positionOnLeft[1]-halfWindow:positionOnLeft[1]+halfWindow]
    #plt.figure()
    #plt.imshow(template)
    
    span = rightGrayImg[positionOnLeft[0]-halfWindow:positionOnLeft[0]+halfWindow, :]
    #plt.figure()
    #plt.imshow(span)
    min_place, min_value = scan_line(span, template, positionOnLeft[1]-250, positionOnLeft[1]-50-halfWindow)
    
    disparity = positionOnLeft[1]-min_place
    print("Disparity: ", disparity, "mm")
    
    depth = focalLength*baseline/(disparity)
    #print(f"Depth of pixel [{positionOnLeft[0]},{positionOnLeft[1]}] in mm: {depth}")
    return depth

In [9]:
leftSet = sorted(glob.glob('./../../Data/Stereo_conveyor_without_occlusions_undistorted/left/*.png')) #= cv2.VideoCapture(CamL_id)

#Create Kalman Filter for state prediction
prediction, kalman = Kalman(2,3)
 
#Create Background Removal
fgbg = BkgRmv(sel=1)

#Preprocess image?
morph = True
crop = True

# New Object?
thresholdCenter = 10
prevCenter = (0, 0)
prevRadius = 0
thresholdRadius = 5

# Crop the image?
cropTop = 200
cropBottom = 600
cropLeft = 300
cropRight = 1150

for leftName in leftSet:
   print("Next Frame")
   rightName = leftName.replace('left','right').replace('Left','Right')

   imgleft = cv2.imread(leftName)
   imglft = cv2.cvtColor(imgleft,cv2.COLOR_BGR2GRAY)
   originalFrame = imgleft.copy()

   
   if morph == True:
      # Smoothening of the image 
      # Morphological Operations: Erosion +  Dilation
      kernel = np.ones((5,5), np.uint8) 
      imglft = cv2.morphologyEx(imgleft,cv2.MORPH_OPEN,kernel)  #Open = Erosion + Dilation
   
   filtered = fgbg.apply(imglft)
   filtered = cv2.morphologyEx(filtered,cv2.MORPH_OPEN,kernel)

   if crop == True:
      #Focus the analysis in the area of interest
      cropped = filtered[cropTop:cropBottom, cropLeft:cropRight]

   cnts, _ = cv2.findContours(cropped.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
   if len(cnts) > 0:
      # In case contours were found..
      # Find the largest one..
      c = max(cnts, key=cv2.contourArea)
      ((x, y), radius) = cv2.minEnclosingCircle(c)
      M = cv2.moments(c) #Find the center of the shape
      if M["m00"] == 0.0:
                continue #If the center of the biggest object in the filtered image is in the origin stop
      #Recovering of the global positions
      cX = int(M["m10"] / M["m00"]) + cropLeft 
      cY = int(M["m01"] / M["m00"]) + cropTop
      center = (cX,cY)
      #print(radius)
      if radius != prevRadius:#radius > 40 and radius < 200:#and radius + thresholdRadius >= prevRadius:
      # If the object meets the size requirements...
         cv2.circle(originalFrame, (int(x)+cropLeft, int(y)+cropTop), int(radius),
                    (0, 255, 255), 2)                             # Enclsoing circle
         cv2.circle(originalFrame, center, 5, (0, 0, 255), -1)   # Center of mass
         
         #In order to compute the depth, we will use the stereo pair
         #Then this depth will be used by the kalman filter for 3D state estimation
         print("Computing depth")
         imgright = cv2.imread(rightName)
         imrght = cv2.cvtColor(imgright, cv2.COLOR_BGR2GRAY)
         depth = Depth(originalFrame, imrght, center, 50) # We pass the center of gravity of the object to the estimate it's depth
         print(depth)
         
         cv2.putText(originalFrame, f"Depth: {depth:.2f} mm", (0,200), cv2.FONT_ITALIC, 1, (0,0,0))
         
         prevRadius = radius
         text = "Object found at:"
         colour = (0, 255, 0)
         cv2.putText(originalFrame,"({}, {})".format(int(center[0]), int(center[1])), (0, 100), cv2.FONT_ITALIC, 1, colour)

         #Once that the object has been found we start the correction phase of the Kalman filter
         kalman.correct(np.array([np.float32(center[0]), np.float32(center[1]), np.float32(depth)], np.float32))
         prevCenter = center
      else: 
         # Object not found
         text = "Object not found"
         objectFound = False
         colour = (0, 0, 255)
         prevRadius = 0
         cv2.putText(originalFrame, text, (0, 100), cv2.FONT_ITALIC, 1, colour)
   
   else:
   #We will only make predictions when the object is not found
      prediction = kalman.predict()
      center_pred = (int(prediction[0]+prediction[1]),int(prediction[2]+prediction[3]))
      
      cv2.circle(originalFrame, center_pred, 5, (255, 0, 0), -1)
      cv2.putText(originalFrame,"({}, {}, {})".format(prediction[0], prediction[2], prediction[4]), 
                     (0, 150), cv2.FONT_ITALIC, 1, colour)
      
         
   cv2.imshow('Video', originalFrame)
   if cv2.waitKey(30) & 0xFF == ord('q'):
      cv2.destroyAllWindows()
      break
         

BackGroundSubstractor Gaussian mixture model
Next Frame
469.0422058105469
Next Frame
4.609872341156006
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
4.94984769821167
Next Frame
3.6056511402130127
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
3.201662063598633
Next Frame
Next Frame
Next Frame
Next Frame
3.9052248001098633
Next Frame
3.201662063598633
Next Frame
Next Frame
Next Frame
3.201662063598633
Next Frame
5.147915363311768
Next Frame
3.9052248001098633
Next Frame
16.19423484802246
Next Frame
28.160354614257812
Next Frame
24.253965377807617
Next Frame
26.669370651245117
Next Frame
9.708344459533691
Next Frame
14.30044937133789
Next Frame
8.860122680664062
Next Frame
6.726912021636963
Next Frame
6.103377819061279
Next Frame
Next Frame
Next Frame
2.8285269737243652
Next Frame
9.617792129516602
Next Frame
24.15066909790039
Next Frame
18.33721923828125
Next Frame
17.219274520874023
Next Frame
20.359373092651367
Next Frame
15

KeyboardInterrupt: 